In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm 
import statsmodels.formula.api as smf
from statsmodels.iolib.summary2 import summary_params # вывод результатов тестирования
from statsmodels.iolib.summary2 import summary_col # вывод результатов тестирования
from statsmodels.stats.outliers_influence import variance_inflation_factor # VIF
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/ryupepa/Econometrica/master/python-notebooks/data-csv/sleep75.csv')
df['totwrk_south']=df['totwrk']*df['south']
df

,age,black,case,clerical,construc,educ,earns74,gdhlth,inlf,leis1,...,totwrk,union,worknrm,workscnd,exper,yngkid,yrsmarr,hrwage,agesq,totwrk_south
0,32,0,1,0.000000,0.000000,12,0,0,1,3529,...,3438,0,3438,0,14,0,13,7.070004,1024,0
1,31,0,2,0.000000,0.000000,14,9500,1,1,2140,...,5020,0,5020,0,11,0,0,1.429999,961,5020
2,44,0,3,0.000000,0.000000,17,42500,1,1,4595,...,2815,0,2815,0,21,0,0,20.529997,1936,0
3,30,0,4,0.000000,0.000000,12,42500,1,1,3211,...,3786,0,3786,0,12,0,12,9.619998,900,0
4,64,0,5,0.000000,0.000000,14,2500,1,1,4052,...,2580,0,2580,0,44,0,33,2.750000,4096,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
701,45,0,702,0.182331,0.030075,12,5500,1,0,5069,...,2026,0,2026,0,27,0,18,NaN,2025,0
702,34,0,703,0.182331,0.030075,10,2500,0,0,5885,...,675,1,465,210,18,0,4,NaN,1156,0
703,37,0,704,0.182331,0.030075,12,3500,1,0,4719,...,1851,0,1851,0,19,0,17,NaN,1369,0
704,54,0,705,0.182331,0.030075,17,32500,1,0,5149,...,1961,1,1481,480,31,0,22,NaN,2916,1961


In [5]:
# Модель sleep на totwrk/100, totwrk^2/10000, age, smsa, male
Sleep_eq12 = smf.ols(formula = 'sleep~totwrk +  age + totwrk_south+ smsa + male + south', data = df).fit()


In [6]:
info_dict = {'No. Observations': lambda x: '{0:d}'.format(int(x.nobs)), 
             'Residual Std. Error': lambda x: "%#8.3f" % x.mse_resid**0.5,
             'F-statistic': lambda x: '{:.3f}'.format(x.fvalue), 
             'Prob (F-statistic)': lambda x: '{:.3f}'.format(x.f_pvalue)
             }
print(summary_col(Sleep_eq12, float_format='%.3f', stars=True, info_dict = info_dict))


                       sleep   
-------------------------------
Intercept           3484.252***
                    (70.188)   
totwrk              -0.178***  
                    (0.019)    
age                 2.908**    
                    (1.386)    
totwrk_south        0.064      
                    (0.050)    
smsa                -56.929*   
                    (32.909)   
male                91.015***  
                    (34.186)   
south               -41.027    
                    (118.036)  
R-squared           0.133      
R-squared Adj.      0.125      
No. Observations    706        
Residual Std. Error 415.690    
F-statistic         17.799     
Prob (F-statistic)  0.000      
Standard errors in parentheses.
* p<.1, ** p<.05, ***p<.01


In [7]:
df = summary_params(Sleep_eq12, alpha=0.05)
df['significance'] = df.apply(lambda x: 'Значим' if x['P>|t|']<0.05 else 'Незначим', axis=1)
df

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],significance
Intercept,3484.251830,70.188075,49.641649,2.396751e-231,3346.447119,3622.056540,Значим
totwrk,-0.178110,0.018965,-9.391382,8.193899e-20,-0.215346,-0.140875,Значим
age,2.908394,1.385722,2.098829,3.619016e-02,0.187717,5.629071,Значим
totwrk_south,0.063627,0.050001,1.272498,2.036193e-01,-0.034544,0.161798,Незначим
smsa,-56.929434,32.908726,-1.729919,8.408614e-02,-121.541227,7.682359,Незначим
male,91.015100,34.185730,2.662371,7.937669e-03,23.896084,158.134117,Значим
south,-41.026786,118.036491,-0.347577,7.282624e-01,-272.775332,190.721760,Незначим
